# Sprint 12 - Lógica Difusa (Sesiones)
**Versión para estudiantes**

En este caso vamos a conocer cómo funciona el aprendizaje computacional basado en **lógica difusa**. Para comprender su utilidad, pensemos que no siempre es fácil contar con datos exactos y cuantificables de variables a utilizar en nuestros modelos predictivos, ya sea por dificultad para accederlos, porque los mismos se generan al mismo momento en que se requiere una predicción, o simplemente por que no existe una representación numérica objetiva para gestionarlos. Aquí algunas situaciones ilustrativas al respecto:

* Los mecanismos inteligentes de calefacción aprenden a establecer la temperatura de una casa o departamento en base a la información recogida de las personas que allí viven. Sin embargo, esta tecnología rara vez cuenta con especificaciones exactas asociadas a temperatura o humedad. Por el contrario, suele recibir datos por mensajes de voz con términos como "hace mucho calor/frio", o "baja un poco la calefacción". 
* El sistema automático de frenado de los vehículos depende de la velocidad y de la distancia a la cual se encuentra el vehículo de un obstáculo. Su activación y nivel de frenado deben ser instántaneos por lo que no resulta práctico ni seguro contar con un modelo supervisado tradicional que deba procesar toda la información histórica, entrenarse y generar un pronóstico para tomar decisiones. 
* La decisión sobre la dosificación de ciertas medicinas para controlar sintomas o enfermedades en muchos casos depende de criterios semánticos y experiencia previa del especialista. Por ejemplo, se suelen emplear términos como fiebre "alta" o "baja", pero no se tiene una magnitud específica de temperatura corporal que describa con certeza estas clasificaciones y que puedan incorporarse en modelos predictivos tradicionales.

La lógica difusa por tanto permite generar pronósticos rápidos y concretos mediante herramientas llamadas *controladores*, que buscan aproximar el razonamiento humano en base al establecimiento de criterios probabilísticos y reglas semánticas. Por lo anterior, resulta un método extremadamente versatil que puede aplicarse en diversos ámbitos como la robótica, seguridad, leyes, arte, domótica, y por supuesto, en los negocios.

Visto esto, a continuación mostraremos cómo implementar uno de estos controladores en Python para establecer una política de descuentos inteligente en una empresa.

## Entendimiento del contexto

La gestión de ingresos es un pilar fundamental para la rentabilidad en todas las empresas de transporte aéreo de pasajeros, especialmente porque el sector opera con márgenes muy estrechos, alta competencia y una capacidad fija que se deprecia rápidamente (los asientos no vendidos se pierden para siempre). Esta etrategia de negocios se centra en maximizar el ingreso por asiento disponible, administrando el precio en función de diversas variables de mercado como la antelación de compra, la demanda esperada y el estado actual de disponibilidad de espacios en cada vuelo. Como resultado, se genera una menor probabilidad de venta de tickets baratos, así como una optimización de la ocupación efectiva sin la necesidad de incurrir en mayores costos.

En este sentido, una aerolínea emergente que ofrece vuelos diarios en una frecuencia otorgada apenas el año anterior, te ha contratado para que le asistas en el desarrollo de un sistema inteligente de descuento a ser utilizado tanto a nivel de ventas directas como en su página web. Para esto, el vicepresidente de ventas te comenta que el descuento deseado depende exclusivamente de tres variables:

* La temporada del año en la que se realiza el viaje.
* La disponibilidad efectiva de cada uno de los vuelos diarios.
* Los días de antelación de compra por parte de los clientes.

Adicionalmente, te advierte que existen los siguientes inconvenientes a tener en cuenta:

1. Al haber sido operada esta frecuencia solamente durante un año, aún no se tiene una certeza absoluta respecto a cuándo son los meses de alta demanda (en los que los descuentos deberían ser menores) y baja demanda (en los que los descuentos deberían ser mayores).
2. Si bien es lógico suponer que a mayor disponibilidad debería existir un mayor descuento con el propósito de llenar los vuelos, no se tiene claridad sobre en qué porcentaje de ocupación hay que activar políticas promocionales.
3. Lo mismo sucede con los días de antelación de compra, se supone que a menos días de antelación habrán mayores descuentos, pero no es claro en que día precisamente aplicarlos.
4. Finalmente, si bien los descuentos no pueden superar el 50%, se desconoce el valor exacto a considerar en distintos escenarios y combinaciones de estas variables.

Tu objetivo entonces es dar solución a esta problemática considerando estos antecedentes.

## Entendimiento de los datos

Carga las librerías con las que vas a trabajar incluyendo el módulo *skfuzzy.control* de la librería **Scikit - Fuzzy**. Esta librería cuenta con herramientas para desarrollar modelos de lógica difusa.

El vicepresidente comercial comparte contigo el archivo **lineamientos_desc.xlsx** que contiene los 45 lineamientos generales de descuento que están siendo aplicados por el equipo de ventas de la aerolínea de manera referencial. 

Carga y explora estos datos.

Como puedes evidenciar, los lineamientos dan cuenta de los problemas antes expuestos. En todo caso son importantes en cuanto a que establecen los criterios fundamentales que deberían ser considerados por un modelo de pronóstico de descuentos. Llamémoslos entonces **reglas difusas**. 

Ante esta falta de información, tu decides llevar a cabo una encuesta en la que consultas a todos los ejecutivos comerciales respecto a lo siguiente:

1. Sobre temporadas del año:
    - ¿Qué meses consideras que son temporadas peak?
    - ¿Qué meses consideras que son temporadas regulares?
    - ¿Qué meses consideras que son temporadas low?
2. Sobre niveles de disponibilidad:
    - ¿Entre qué porcentajes consideras que hay una disponibilidad baja?
    - ¿Entre qué porcentajes consideras que hay una disponibilidad media?
    - ¿Entre qué porcentajes consideras que hay una disponibilidad alta?
3. Sobre los niveles de descuento:
    - ¿Entre qué porcentajes consideras que un descuento es bajo?
    - ¿Entre qué porcentajes consideras que un descuento es moderado?
    - ¿Entre qué porcentajes consideras que un descuento es alto?
    - ¿Entre qué porcentajes consideras que un descuento es muy alto?

Las respuestas obtenidas son procesadas y los resultados pueden ser vistos en el archivo **conj_difusos.xlsx**.

Carga y explora estos datos que se encuentran en las cuatro pestañas del archivo.

Vale indicar que los valores aquí representados se interpretan como el porcentaje de ejecutivos comerciales que respondieron afirmativamente a la pregunta correspondiente. Así por ejemplo:

* Un 40% de los encuestados cree que febrero es un mes de temporada "peak".
* Un 70% considera que una disponibilidad del vuelo de 63% equivale a un nivel "medio".
* Un 60% considera que un descuento de 13% es "alto".

A todos estos atributos obtenidos los llamaremos de aquí en adelante **conjuntos difusos** o **membresías**. 

## Modelo de control difuso

Antes de continuar con el desarrollo de nuestro caso conviene explicar en mayor detalle cómo funcionan los modelos de control difuso. En primer lugar se necesita tener claridad que al igual que un modelo supervisado, su propósito es predecir la variable objetivo $y$ en función de atributos $X$ tal que

$$ y = f(X) $$

La gran diferencia ahora radica en que tanto $y$ como $X$ no corresponden a variables exactas o cuantificables, sino que están asociadas a los ya mencionados **conjuntos difusos**. Adicionalmente, la relación funcional descrita no es estrictamente algebráica, sino que responde a una lógica particular del contexto dada por las **reglas difusas**. Definamos entonces a la combinación de los conjuntos y las reglas difusas como nuestra **base de conocimiento**.

Visto esto, en el siguiente gráfico se muestra la idea conceptual detrás de estos modelos:

![](controlador_difuso.png)

A modo de simil, la base de conocimiento actúa como nuestro ya conocido conjunto de entrenamiento, el cual sirve como insumo para un algoritmo desarrollado por Ebhasim Mamdani. Una vez entrenado, este algoritmo puede recibir un escenario concreto de los atributos $X$ para generar una predicción de $y$.

¿Qué sucede dentro del algoritmo de Mamdani? En este momento no nos vamos a preocupar de las justificaciones teóricas que lo sustentan, pero a continuación explicamos el proceso realizado tomando como ejemplo un escenario en el mes de octubre, con 50 días de anticipación y un porcentaje de disponibilidad del 31%:

1. *Establecer los conjuntos difusos que se activan en el escenario (aquellos con un valor de membresía mayor a 0).*

Quedan entonces activos:

* Para temporada: "regular" y "low".
* Para dias de anticipación: "30_60".
* Para disponibilidad: "media".

2. *Definir las reglas que se activan por estos conjuntos (aquellas que tienen una de las opciones anteriores).*

Quedan entonces activas las reglas 20 y 35.

3. *Establecer un valor referencial $\eta_r$ para cada una de las reglas activas $r$ tal que* $\eta_r = \min{(t_r, d_r, l_r)}$*, donde* 

    * $t_r$ *es el valor de membresía de la temporada en la regla.* 
    * $d_r$ *es el valor de membresía de los dias de anticipacion en la regla.* 
    * $l_r$ *es el valor de membresía del nivel de disponibilidad en la regla.*

4. *Recalcular los valores de membresía $\gamma$ de los conjuntos difusos $C$ de los niveles de descuento "moderado" y "alto" tal que*

$$ \gamma^{(n)} = \min{(\gamma, \eta_r)}\quad \forall \gamma \in C  $$

5. *Generar un conjunto difuso final $M$ tal que cada uno de sus elementos $\mu$ esté definido por*

$$ \mu = \max{\left(\gamma_{moderado}^{(n)},\gamma_{alto}^{(n)}\right)}\quad \forall \mu \in M $$

6. *Calcular el porcentaje resultante $p$ como un valor central del conjunto difuso final $M$ tal que*

$$ p = \frac{\int_{M} \mu p_{\mu}}{\int_{M} \mu} \approx \frac{\sum_M \mu p_\mu}{\sum_M \mu}$$

*donde $p_\mu$ es el porcentaje de descuento asociado a $\mu$.*

## Ingeniería de datos

Visto el proceso anterior, la librería **Scikit - Fuzzy** nos permite desarrollar este modelo de una forma más directa. Para utilizarla en todo caso, es necesario preparar estos datos tal que Python comprenda que estamos trabajando con conjuntos y reglas difusas.

Define entonces los atributos como conjuntos difusos mediante la función `ctrl.Antecedent`.

Define ahora a la variable objetivo como un conjunto difuso mediante la función `ctrl.Consequent`.

Ingresa ahora los valores de membresías para estos conjuntos difusos. Para esto considera que los conjunto definidos son similares a diccionarios.

Define finalmente las reglas difusas para completar nuestra base de conocimiento. Utiliza la función `ctrl.Rule` y guíate por el siguiente ejemplo para la regla 1:

```py
rule_01 = ctrl.Rule(x_temp["peak"] & x_dias["00_30"] & x_disp["baja"], y_desc["moderado"])
```

## Creación de modelo de control difuso

Crea el controlador a través de la función `ctrl.ControlSystem`. Ingresa como argumentos todas las reglas de la base de conocimiento como parte de una lista.

A partir del controlador, crea también un simulador difuso `ctrl.ControlSystemSimulation`. Este simulador permitirá ejecutar los escenarios con el algoritmo de Mamdani. 

Veamos si nuestro modelo funciona. Simula el escenario del ejemplo realizado en secciones precedentes:

* mes: 10
* dias de antelación: 50.
* nivel de disponibilidad: 31%

Utiliza para esto los métodos `input` y `compute` y `output` del simulador.

Presenta el resultado de una forma más visual con el método `view` del conjunto difuso del nivel de descuento.

## Automatización del modelo

Hagamos ahora que nuestro modelo pueda pronosticar el porcentaje de descuento para distintos escenarios de manera rápida. Crea una función que reciba como entradas valores de atributos y devuelva el valor de descuento calculado. 

¡Buen trabajo! La aerolinea ya cuenta con una herramienta objetiva y eficiente que le permite saber qué descuento aplicar ante distintos escenarios. Para hacerte una idea más general de su funcionalidad, en el siguiente código puedes evidenciar cómo evolucionan los descuentos calculados por el modelo ante distintos escenarios de meses y porcentajes de disponibilidad.

```py
# Generar descuentos ante distintos escenarios
mes_lista = []
disp_lista = []
desc_lista = []

for m in x_temp.universe:
    for d in x_disp.universe:
        mes_lista.append(m)
        disp_lista.append(d)
        desc_lista.append(fun_descuento(m, 90, d))

df_resultado = pd.DataFrame(dict(
    mes = mes_lista,
    disponibilidad = disp_lista,
    descuento = desc_lista
))

# Crear variables agregadas
df_resultado["trimestre"] = (df_resultado["mes"] - 1) // 3 + 1
df_resultado["grupo_disponibilidad"] = ((df_resultado["disponibilidad"] + 1) // 5) * 5

# Visualizar descuentos calculados
df_resultado.pivot_table(
    index = "grupo_disponibilidad",
    columns = "trimestre",
    values = "descuento",
    aggfunc = "median"
).style.background_gradient(cmap = "Blues", axis = None)
```